anytimeのスクレイピング

In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd 
import numpy as np

ジム1件の情報を取得する。

In [2]:
base_url = "https://www.anytimefitness.co.jp/kanto/tokyo/" 
response = requests.get(base_url)
soup     = BeautifulSoup(response.content, "lxml")
items    = soup.find(class_="areasort")

In [3]:
# スクレイピングした HTML情報出力,取得したデータをテキストファイルに保存するための手順
#f = open("test2.txt", "w", encoding="utf-8") #エラーが出たためencoding="utf-8"を追加
#f.write(str(items))
#f.close()

In [4]:
# 各情報の取得
#店舗名
gym_name     = items.find(class_="name").get_text() if items.find(class_="name") else None

#住所
gym_address    = items.find(class_="address").get_text() if items.find(class_="address") else None

#店舗情報リンク
link_element = items.find('a', href=True)
relative_url = link_element['href'] if link_element else None
base_url = "https://www.anytimefitness.co.jp"
gym_link = base_url + relative_url if relative_url else None


print("店舗名 (Name):", gym_name)
print("住所 (Address):", gym_address)
print("リンク (Link):", gym_link)

店舗名 (Name): 秋葉原店
住所 (Address): 〒101-0021 東京都千代田区外神田2-5-12 タカラビル1F-B1
リンク (Link): https://www.anytimefitness.co.jp/akb/


東京都全店舗の情報を取得する。

In [5]:
# 基本URLの設定
url = "https://www.anytimefitness.co.jp/kanto/tokyo/"

all_data = []

response = requests.get(url)
soup = BeautifulSoup(response.content, 'lxml')
items = soup.findAll("div", {"class": "areasort"})

print("items", len(items))

for item in items:
    base_data = {}

    # 'areasort' クラスを持つ 'div' 要素内のすべての 'li' 要素を取得
    tbodys = item.findAll("li")

    for tbody in tbodys:
        data = base_data.copy()

        data["店舗名"] = tbody.select_one("p.name").get_text(strip=True) if tbody.select_one("p.name") else None
        data["住所"] = tbody.select_one("p.address").get_text(strip=True) if tbody.select_one("p.address") else None

        link_element = tbody.select_one('a', href=True)
        relative_url = link_element['href'] if link_element else None
        base_url = "https://www.anytimefitness.co.jp"
        data["リンク"] = base_url + relative_url if relative_url else None

        all_data.append(data)

# 結果を表示
for data in all_data:
    print(data)


items 46
{'店舗名': '秋葉原店', '住所': '〒101-0021 東京都千代田区外神田2-5-12 タカラビル1F-B1', 'リンク': 'https://www.anytimefitness.co.jp/akb/'}
{'店舗名': None, '住所': None, 'リンク': None}
{'店舗名': None, '住所': None, 'リンク': None}
{'店舗名': None, '住所': None, 'リンク': None}
{'店舗名': '内神田店', '住所': '〒101-0047 東京都千代田区内神田1-15-2 神田オーシャンビル B1F', 'リンク': 'https://www.anytimefitness.co.jp/uchikanda/'}
{'店舗名': None, '住所': None, 'リンク': None}
{'店舗名': None, '住所': None, 'リンク': None}
{'店舗名': None, '住所': None, 'リンク': None}
{'店舗名': None, '住所': None, 'リンク': None}
{'店舗名': None, '住所': None, 'リンク': None}
{'店舗名': '御茶ノ水店', '住所': '〒101-0062 東京都千代田区神田駿河台2-1-20 お茶の水ユニオンビル B1F', 'リンク': 'https://www.anytimefitness.co.jp/ochanomizu/'}
{'店舗名': None, '住所': None, 'リンク': None}
{'店舗名': None, '住所': None, 'リンク': None}
{'店舗名': None, '住所': None, 'リンク': None}
{'店舗名': None, '住所': None, 'リンク': None}
{'店舗名': None, '住所': None, 'リンク': None}
{'店舗名': None, '住所': None, 'リンク': None}
{'店舗名': None, '住所': None, 'リンク': None}
{'店舗名': None, '住所': None, 'リンク': None}
{'店舗名': '九段

In [6]:
df = pd.DataFrame(all_data)
df = df.dropna() # Noneデータの削除
df.reset_index(drop=True, inplace=True) # インデックスをリセット
df.head()

,店舗名,住所,リンク
0,秋葉原店,〒101-0021 東京都千代田区外神田2-5-12 タカラビル1F-B1,https://www.anytimefitness.co.jp/akb/
1,内神田店,〒101-0047 東京都千代田区内神田1-15-2 神田オーシャンビル B1F,https://www.anytimefitness.co.jp/uchikanda/
2,御茶ノ水店,〒101-0062 東京都千代田区神田駿河台2-1-20 お茶の水ユニオンビル B1F,https://www.anytimefitness.co.jp/ochanomizu/
3,九段下店,〒102-0072 東京都千代田区飯田橋1-12-12 九段大和エステート 1-3F,https://www.anytimefitness.co.jp/kudanshita/
4,麹町店,〒102-0083 東京都千代田区麹町4-8 麹町クリスタルシティ 1F,https://www.anytimefitness.co.jp/koujimati/


住所から郵便番号を削除

In [7]:
# 郵便番号を削除する関数
def remove_postal_code(address):
    return re.sub(r'〒\d{3}[-‐]\d{4}', '', address).strip()

# 住所から郵便番号を削除
df['住所'] = df['住所'].apply(remove_postal_code)

# 結果の表示
df.head()

,店舗名,住所,リンク
0,秋葉原店,東京都千代田区外神田2-5-12 タカラビル1F-B1,https://www.anytimefitness.co.jp/akb/
1,内神田店,東京都千代田区内神田1-15-2 神田オーシャンビル B1F,https://www.anytimefitness.co.jp/uchikanda/
2,御茶ノ水店,東京都千代田区神田駿河台2-1-20 お茶の水ユニオンビル B1F,https://www.anytimefitness.co.jp/ochanomizu/
3,九段下店,東京都千代田区飯田橋1-12-12 九段大和エステート 1-3F,https://www.anytimefitness.co.jp/kudanshita/
4,麹町店,東京都千代田区麹町4-8 麹町クリスタルシティ 1F,https://www.anytimefitness.co.jp/koujimati/


国土地理院APIで緯度経度を取得

In [8]:
import requests
# APIのURL
url = "https://msearch.gsi.go.jp/address-search/AddressSearch?q="

# 住所から緯度経度を取得する関数
def get_lat_lon(address):
    try:
        res = requests.get(url + address)
        res.raise_for_status()  # HTTPエラーが発生した場合に例外を発生させる
        data = res.json()
        if data and len(data) > 0:
            location = data[0]['geometry']['coordinates']
            return location[1], location[0]  # 緯度、経度を返す
        else:
            return None, None
    except Exception as e:
        print(f"Error retrieving location for address {address}: {e}")
        return None, None

# 緯度経度の取得とデータフレームへの保存
df[['緯度', '経度']] = df['住所'].apply(lambda x: pd.Series(get_lat_lon(x)))

# 結果の表示
df.head()

,店舗名,住所,リンク,緯度,経度
0,秋葉原店,東京都千代田区外神田2-5-12 タカラビル1F-B1,https://www.anytimefitness.co.jp/akb/,35.700256,139.769089
1,内神田店,東京都千代田区内神田1-15-2 神田オーシャンビル B1F,https://www.anytimefitness.co.jp/uchikanda/,35.691555,139.765762
2,御茶ノ水店,東京都千代田区神田駿河台2-1-20 お茶の水ユニオンビル B1F,https://www.anytimefitness.co.jp/ochanomizu/,35.699497,139.762421
3,九段下店,東京都千代田区飯田橋1-12-12 九段大和エステート 1-3F,https://www.anytimefitness.co.jp/kudanshita/,35.698547,139.748947
4,麹町店,東京都千代田区麹町4-8 麹町クリスタルシティ 1F,https://www.anytimefitness.co.jp/koujimati/,35.684250,139.736771


In [9]:
# 住所から「〇〇区」を抽出する関数
def extract_ku(address):
    match = re.search(r'東京都(\w+区)', address)
    return match.group(1) if match else None

# 新しいカラム「区」を作成
df['区'] = df['住所'].apply(extract_ku)

# 結果の表示
df.head()

,店舗名,住所,リンク,緯度,経度,区
0,秋葉原店,東京都千代田区外神田2-5-12 タカラビル1F-B1,https://www.anytimefitness.co.jp/akb/,35.700256,139.769089,千代田区
1,内神田店,東京都千代田区内神田1-15-2 神田オーシャンビル B1F,https://www.anytimefitness.co.jp/uchikanda/,35.691555,139.765762,千代田区
2,御茶ノ水店,東京都千代田区神田駿河台2-1-20 お茶の水ユニオンビル B1F,https://www.anytimefitness.co.jp/ochanomizu/,35.699497,139.762421,千代田区
3,九段下店,東京都千代田区飯田橋1-12-12 九段大和エステート 1-3F,https://www.anytimefitness.co.jp/kudanshita/,35.698547,139.748947,千代田区
4,麹町店,東京都千代田区麹町4-8 麹町クリスタルシティ 1F,https://www.anytimefitness.co.jp/koujimati/,35.684250,139.736771,千代田区


データベースの作成

In [10]:
import sqlite3

In [11]:
# データベース接続を作成
conn = sqlite3.connect('property.db')

In [12]:
# データフレームをSQLiteテーブルに変換
df.to_sql('anytime', conn, if_exists='replace', index=False)

245

In [13]:
conn.close()